In [27]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_columns', 500)

In [ ]:
new_data = pd.read_stata('12cdenhancedupdate1019_clean.dta')

In [168]:
df = new_data[['precinct', 'voters_age', 'voters_birthdate', 'primary_20180626', 'primary_20180913', 'special_20190226', 'primary_20190625']]

In [169]:
df['under_30'] = df.voters_age.apply(lambda x: 1 if x<30 else 0)
df['under_60'] = df.voters_age.apply(lambda x: 1 if x<60 else 0)

In [170]:
elections = ['special_20190226', 'primary_20190625', 'primary_20180913', 'primary_20180626']
# look at 6-25 multiple columns to see if there's a difference
# add columns for 2 and not 1
# add column for first vote in 6-25
# 4x voters
# final data frame -- rows will be election districts, columns = total voters from other spreadsheet, number voters
# in all 4 elections, number under 30 voters, number under 60 voters, average age in each election, 

In [171]:
for election in elections:
    df[election] = df[election].apply(lambda x: 1 if x=='Y' else 0)
    df[election + "date"] = pd.to_datetime(election[-8:], format='%Y%m%d')

In [172]:
df['KEY'] = df.precinct.apply(lambda x: int(x[-2:] + x[-6:-3]))

In [173]:
df['voters_birthdate'] = pd.to_datetime(df['voters_birthdate'])

In [174]:
for election in elections:
    df["age_in_"+election] = round((df[election+'date']-df['voters_birthdate']).astype(int)/(60*60*24*365*1000000000),2)

In [175]:
df['four_x'] = df[elections[0]]*df[elections[1]]*df[elections[2]]*df[elections[3]]

In [176]:
df['skipped_patel'] = ((df[elections[2]]==1) & (df[elections[3]]==0)).astype(int)

In [177]:
df['new_voter'] = ((df[elections[1]]==1) & (df[elections[0]]==0) & (df[elections[2]]==0) & (df[elections[3]]==0)).astype(int)

In [178]:
df['four_x_under_30'] = df.four_x * df.under_30
df['four_x_under_60'] = df.four_x * df.under_60

In [179]:
df.drop(['precinct', 'voters_age', 'voters_birthdate', 'special_20190226date', 'primary_20190625date', 'primary_20180913date', 'primary_20180626date'], axis=1, inplace=True)

In [180]:
df.rename(columns={"primary_20180626": "06-26-2018", "primary_20180913": "09-13-2018", "special_20190226": "02-26-2019", 'primary_20190625': '06-25-2019', "age_in_primary_20180626": "age_in_06-26-2018", "age_in_primary_20180913": "age_in_09-13-2018", "age_in_special_20190226": "age_in_02-26-2019", 'age_in_primary_20190625': 'age_in_06-25-2019'}, inplace=True)

In [181]:
cols = ['KEY', '06-26-2018', '09-13-2018', '02-26-2019', '06-25-2019',
        'age_in_02-26-2019', 'age_in_06-25-2019', 'age_in_09-13-2018', 'age_in_06-26-2018',
        'under_30', 'under_60', 'new_voter', 'four_x', 'four_x_under_30', 'four_x_under_60', 'skipped_patel']
df = df[cols]

In [182]:
sum_group = ['KEY', '06-26-2018', '09-13-2018', '02-26-2019', '06-25-2019', 'under_30', 'under_60', 'new_voter', 'four_x', 'four_x_under_30', 'four_x_under_60', 'skipped_patel']
avg_group = ['KEY', 'age_in_02-26-2019', 'age_in_06-25-2019', 'age_in_09-13-2018', 'age_in_06-26-2018']
sum_df = df[sum_group].groupby(['KEY']).sum()
avg_df = df[avg_group].groupby(['KEY']).mean()

In [183]:
df = sum_df.merge(avg_df, on='KEY')

In [184]:
df

,06-26-2018,09-13-2018,02-26-2019,06-25-2019,under_30,under_60,new_voter,four_x,four_x_under_30,four_x_under_60,skipped_patel,age_in_02-26-2019,age_in_06-25-2019,age_in_09-13-2018,age_in_06-26-2018
KEY,,,,,,,,,,,,,,,
30038,78,107,65,85,21,124,13,24,2,20,47,45.800309,46.126235,45.345679,45.128704
30056,80,100,58,106,13,125,21,30,2,24,36,45.672774,45.998710,45.218129,45.001161
30064,30,56,27,56,16,64,12,11,1,11,32,46.206163,46.531395,45.750581,45.534651
30066,76,109,70,116,22,123,18,31,3,26,45,45.761104,46.086442,45.305890,45.089816
30067,58,83,55,77,20,109,16,18,1,15,36,46.135646,46.462177,45.681361,45.464490
30068,74,132,70,127,29,153,22,25,5,18,66,46.182362,46.508643,45.727487,45.510804
30069,88,119,57,100,19,112,15,25,2,15,44,48.212663,48.536391,47.760296,47.545266
30078,25,56,39,63,17,71,11,9,1,9,35,41.923412,42.249412,41.468471,41.252588
30079,84,94,65,75,23,136,12,25,4,22,32,43.994788,44.321152,43.540182,43.323394
